# Multi-turn Example

In [1]:
import nest_asyncio
from ordering2 import MenuPipeline
import ordering2
from gotaglio.gotag import app_configuration, Gotaglio

# Necessary for running asyncio in Jupyter
nest_asyncio.apply()

In [2]:
app_configuration["model_config_file"] = "../../models.json"
app_configuration["model_credentials_file"] = "../../.credentials.json"
gt = Gotaglio([ordering2.MenuPipeline])

In [3]:
runlog1 = gt.run(
    "menu2",
    "cases3.json",
    {
        "prepare.template": "template.txt",
        "infer.model.name": "gpt4o",
    },
    save=True,
)

              Summary for              
 e3637d26-3d41-4ed0-af43-6b39c7e5fe43  
┏━━━━━━━┳━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━┓
┃    id ┃ run      ┃ score ┃ keywords ┃
┡━━━━━━━╇━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━┩
│ 0e6.0 │ COMPLETE │  0.00 │          │
│ c47.0 │ COMPLETE │  2.00 │          │
│ c47.1 │ COMPLETE │  0.00 │          │
│ 894.0 │ COMPLETE │  0.00 │          │
│ adf.0 │ COMPLETE │  0.00 │          │
│ adf.1 │ COMPLETE │  0.00 │          │
│ 7ee.0 │ COMPLETE │  0.00 │          │
└───────┴──────────┴───────┴──────────┘

Total: 7
Complete: 7/7 (100.00%)
Error: 0/7 (0.00%)
Passed: 6/7 (85.71%)
Failed: 1/7 (14.29%)


Results written to logs\e3637d26-3d41-4ed0-af43-6b39c7e5fe43.json


In [4]:
gt.format(runlog1)

## Run: e3637d26-3d41-4ed0-af43-6b39c7e5fe43
## Case: 0e6.0
**PASSED**  

Keywords:   
Complete menu tokens: 1296  
Input tokens: 517, output tokens: 71

**system:**
```json
Your job is to convert text-based restaurant orders into JSON data structures.
You never return a text answer. You always generate a JSON data structure.
The JSON should conform to the following typescript type definition for type `Cart`:

~~~typescript
type Cart={items:Item[]};
type Item=WiseguyMeal<ComboSizes>|Meal<ComboSizes>|Wiseguy|ComboTwo|ComboThree|FountainDrink<any,any>;
type WiseguyMeal<SIZE extends ComboSizes>={name:"Wiseguy 
Meal",size:SIZE,sandwich:Wiseguy|CHOOSE,fries:CHOOSE,drink:ChooseDrink<SIZE>};
type ComboSizes="Large"|CHOOSE;
type CHOOSE="CHOOSE";
type Wiseguy=GenericWiseguy<"Wiseguy"|"Vegan Wiseguy"|"Double Wiseguy"|"Triple Wiseguy"|"Down East Wiseguy">;
type GenericWiseguy<NAME>={name:NAME,type:"Regular"|"With Bacon"|"With Cheese"|"With Bacon and Cheese"|CHOOSE};
type ChooseDrink<SIZE extends DrinkSizes>=FountainDrink<any,SIZE>|CHOOSE;
type DrinkSizes="Large"|CHOOSE;
type FountainDrink<NAME extends DrinkNames,SIZE extends DrinkSizes>={name:NAME,size:SIZE};
type DrinkNames="Coca-Cola"|"Diet Coke"|"Coca-Cola Zero Sugar";
type Meal<SIZE extends 
ComboSizes>={name:"Meal",size:SIZE,sandwich:Wiseguy|CHOOSE,fries:CHOOSE,drink:ChooseDrink<SIZE>};
type ComboTwo={name:"Twofer Combo",one:TwoThreeChoices,two:TwoThreeChoices};
type TwoThreeChoices=Wiseguy|FountainDrink<"Coca-Cola"|"Diet Coke"|"Dr. Pepper"|"Sprite","Medium">|CHOOSE;
type ComboThree={name:"Threefer Combo",one:TwoThreeChoices,two:TwoThreeChoices,three:TwoThreeChoices};
~~~

* Only use CHOOSE when the user didn't specify a choice and there is no template literal contraining the choice.
* If request doesn't match the schema exactly, choose the closest matching item that is strictly legal for the 
schema.
* Combos that don't specify enough items should use CHOOSE instead of duplicating an item.
* Return the JSON object of type `Cart`.
```

**assistant:**
```json
{
  "items": []
}
```

**user:** _a large wiseguy meal with coke_

**assistant:**
```json
{
  "items": [
    {
      "name": "Wiseguy Meal",
      "size": "Large",
      "sandwich": "CHOOSE",
      "fries": "CHOOSE",
      "drink": {
        "name": "Coca-Cola",
        "size": "Large"
      }
    }
  ]
}
```

**No repairs**

**Full query**:
* a large wiseguy meal with coke

## Case: c47.0
**FAILED:** cost=2  

Keywords:   
Complete menu tokens: 1296  
Input tokens: 249, output tokens: 54

**system:**
```json
Your job is to convert text-based restaurant orders into JSON data structures.
You never return a text answer. You always generate a JSON data structure.
The JSON should conform to the following typescript type definition for type `Cart`:

~~~typescript
type Cart={items:Item[]};
type Item=Burger|ComboTwo;
type Burger=GenericBurger<"Bacon Double Cheeseburger"|"Bacon Cheeseburger"|"Double Cheeseburger"|"Cheeseburger">;
type GenericBurger<NAME>={name:NAME};
type ComboTwo={name:"Twofer Combo",one:TwoThreeChoices,two:TwoThreeChoices};
type TwoThreeChoices=GenericBurger<"Bacon Cheeseburger">|CHOOSE;
type CHOOSE="CHOOSE";
~~~

* Only use CHOOSE when the user didn't specify a choice and there is no template literal contraining the choice.
* If request doesn't match the schema exactly, choose the closest matching item that is strictly legal for the 
schema.
* Combos that don't specify enough items should use CHOOSE instead of duplicating an item.
* Return the JSON object of type `Cart`.
```

**assistant:**
```json
{
  "items": []
}
```

**user:** _can I get a cheeseburger twofer_

**assistant:**
```json
{
  "items": [
    {
      "name": "Twofer Combo",
      "one": {
        "name": "Cheeseburger"
      },
      "two": {
        "name": "Cheeseburger"
      }
    }
  ]
}
```

**Repairs:**
* 0:Twofer Combo: change one to `{'name': 'Bacon Cheeseburger'}`
* 0:Twofer Combo: change two to `CHOOSE`

**Full query**:
* can I get a cheeseburger twofer

## Case: c47.1
**PASSED**  

Keywords:   
Complete menu tokens: 1296  
Input tokens: 503, output tokens: 86

**system:**
```json
Your job is to convert text-based restaurant orders into JSON data structures.
You never return a text answer. You always generate a JSON data structure.
The JSON should conform to the following typescript type definition for type `Cart`:

~~~typescript
type Cart={items:Item[]};
type Item=PattyMelt|Burger|KoreanChicken|ComboTwo|ComboThree|FountainDrink<any,any>;
type PattyMelt={name:"Bacon Melt"|"Mushroom and Swiss Melt",options?:(Bacon|Condiments)[]};
type Bacon={amount:Optional,name:"Bacon"};
type Optional="Regular";
type Condiments={amount:Amount,name:"Ketchup"};
type Amount="Regular"|"Extra";
type Burger=GenericBurger<"Bacon Double Cheeseburger"|"Bacon Cheeseburger"|"Double Cheeseburger"|"Cheeseburger">;
type GenericBurger<NAME>={name:NAME,options?:(Bacon|Condiments)[]};
type KoreanChicken={name:"Sweet and Spicy Chicken",options?:(Bacon|Condiments)[]};
type ComboTwo={name:"Twofer Combo",one:TwoThreeChoices,two:TwoThreeChoices};
type TwoThreeChoices=GenericBurger<"Bacon Cheeseburger">|FountainDrink<"Coca-Cola"|"Diet Coke"|"Dr. 
Pepper"|"Sprite","Medium">|CHOOSE;
type FountainDrink<NAME extends DrinkNames,SIZE extends DrinkSizes>={name:NAME,size:SIZE};
type DrinkNames="Sprite"|"Sprite Zero";
type DrinkSizes="CHOOSE";
type CHOOSE="CHOOSE";
type ComboThree={name:"Threefer Combo",one:TwoThreeChoices,two:TwoThreeChoices,three:TwoThreeChoices};
~~~

* Only use CHOOSE when the user didn't specify a choice and there is no template literal contraining the choice.
* If request doesn't match the schema exactly, choose the closest matching item that is strictly legal for the 
schema.
* Combos that don't specify enough items should use CHOOSE instead of duplicating an item.
* Return the JSON object of type `Cart`.
```

**assistant:**
```json
{
  "items": [
    {
      "name": "Twofer Combo",
      "one": {
        "name": "Bacon Cheeseburger"
      },
      "two": "CHOOSE"
    }
  ]
}
```

**user:** _can I get that with extra ketchup and a sprite_

**assistant:**
```json
{
  "items": [
    {
      "name": "Twofer Combo",
      "one": {
        "name": "Bacon Cheeseburger",
        "options": [
          {
            "amount": "Extra",
            "name": "Ketchup"
          }
        ]
      },
      "two": {
        "name": "Sprite",
        "size": "Medium"
      }
    }
  ]
}
```

**No repairs**

**Full query**:
* can I get that with extra ketchup and a sprite
* CHOOSE
* Twofer Combo
* Bacon Cheeseburger

## Case: 894.0
**PASSED**  

Keywords:   
Complete menu tokens: 1296  
Input tokens: 434, output tokens: 79

**system:**
```json
Your job is to convert text-based restaurant orders into JSON data structures.
You never return a text answer. You always generate a JSON data structure.
The JSON should conform to the following typescript type definition for type `Cart`:

~~~typescript
type Cart={items:Item[]};
type Item=PattyMelt|Chicken|KoreanChicken|Pitas|Fish|ComboTwo|FountainDrink<any,any>;
type PattyMelt={name:"Mushroom and Swiss Melt"};
type Chicken=GenericChicken<"Grilled Chicken Sandwich"|"Cordon Bleu">;
type GenericChicken<NAME>={name:NAME};
type KoreanChicken={name:"Sweet and Spicy Chicken"|"Seasame Soy Chicken"|"Spicy Garlic Chicken"};
type Pitas={name:"Lemon Chicken Pita"|"Smokey Chicken Pita"|"Tangy Chicken Pita"};
type Fish={name:"Captain Nemo Burger"};
type ComboTwo={name:"Twofer Combo",one:TwoThreeChoices,two:TwoThreeChoices};
type TwoThreeChoices=GenericChicken<"Grilled Chicken Sandwich">|GenericBurger<"Bacon 
Cheeseburger">|FountainDrink<"Coca-Cola"|"Diet Coke"|"Dr. Pepper"|"Sprite","Medium">|CHOOSE;
type GenericBurger<NAME>={name:NAME};
type FountainDrink<NAME extends DrinkNames,SIZE extends DrinkSizes>={name:NAME,size:SIZE};
type DrinkNames="Coca-Cola"|"Diet Coke"|"Coca-Cola Zero Sugar";
type DrinkSizes="CHOOSE";
type CHOOSE="CHOOSE";
~~~

* Only use CHOOSE when the user didn't specify a choice and there is no template literal contraining the choice.
* If request doesn't match the schema exactly, choose the closest matching item that is strictly legal for the 
schema.
* Combos that don't specify enough items should use CHOOSE instead of duplicating an item.
* Return the JSON object of type `Cart`.
```

**assistant:**
```json
{
  "items": []
}
```

**user:** _i want a a grilled chicken and a fish burger and two cokes_

**assistant:**
```json
{
  "items": [
    {
      "name": "Grilled Chicken Sandwich"
    },
    {
      "name": "Captain Nemo Burger"
    },
    {
      "name": "Coca-Cola",
      "size": "CHOOSE"
    },
    {
      "name": "Coca-Cola",
      "size": "CHOOSE"
    }
  ]
}
```

**No repairs**

**Full query**:
* i want a a grilled chicken and a fish burger and two cokes

## Case: adf.0
**PASSED**  

Keywords:   
Complete menu tokens: 1296  
Input tokens: 434, output tokens: 79

**system:**
```json
Your job is to convert text-based restaurant orders into JSON data structures.
You never return a text answer. You always generate a JSON data structure.
The JSON should conform to the following typescript type definition for type `Cart`:

~~~typescript
type Cart={items:Item[]};
type Item=PattyMelt|Chicken|KoreanChicken|Pitas|Fish|ComboTwo|FountainDrink<any,any>;
type PattyMelt={name:"Mushroom and Swiss Melt"};
type Chicken=GenericChicken<"Grilled Chicken Sandwich"|"Cordon Bleu">;
type GenericChicken<NAME>={name:NAME};
type KoreanChicken={name:"Sweet and Spicy Chicken"|"Seasame Soy Chicken"|"Spicy Garlic Chicken"};
type Pitas={name:"Lemon Chicken Pita"|"Smokey Chicken Pita"|"Tangy Chicken Pita"};
type Fish={name:"Captain Nemo Burger"};
type ComboTwo={name:"Twofer Combo",one:TwoThreeChoices,two:TwoThreeChoices};
type TwoThreeChoices=GenericChicken<"Grilled Chicken Sandwich">|GenericBurger<"Bacon 
Cheeseburger">|FountainDrink<"Coca-Cola"|"Diet Coke"|"Dr. Pepper"|"Sprite","Medium">|CHOOSE;
type GenericBurger<NAME>={name:NAME};
type FountainDrink<NAME extends DrinkNames,SIZE extends DrinkSizes>={name:NAME,size:SIZE};
type DrinkNames="Coca-Cola"|"Diet Coke"|"Coca-Cola Zero Sugar";
type DrinkSizes="CHOOSE";
type CHOOSE="CHOOSE";
~~~

* Only use CHOOSE when the user didn't specify a choice and there is no template literal contraining the choice.
* If request doesn't match the schema exactly, choose the closest matching item that is strictly legal for the 
schema.
* Combos that don't specify enough items should use CHOOSE instead of duplicating an item.
* Return the JSON object of type `Cart`.
```

**assistant:**
```json
{
  "items": []
}
```

**user:** _i want a a grilled chicken and a fish burger and two cokes_

**assistant:**
```json
{
  "items": [
    {
      "name": "Grilled Chicken Sandwich"
    },
    {
      "name": "Captain Nemo Burger"
    },
    {
      "name": "Coca-Cola",
      "size": "CHOOSE"
    },
    {
      "name": "Coca-Cola",
      "size": "CHOOSE"
    }
  ]
}
```

**No repairs**

**Full query**:
* i want a a grilled chicken and a fish burger and two cokes

## Case: adf.1
**PASSED**  

Keywords:   
Complete menu tokens: 1296  
Input tokens: 669, output tokens: 110

**system:**
```json
Your job is to convert text-based restaurant orders into JSON data structures.
You never return a text answer. You always generate a JSON data structure.
The JSON should conform to the following typescript type definition for type `Cart`:

~~~typescript
type Cart={items:Item[]};
type Item=PattyMelt|Burger|Chicken|KoreanChicken|Pitas|Fish|ComboTwo|ComboThree|FountainDrink<any,any>;
type PattyMelt={name:"Bacon Melt"|"Mushroom and Swiss Melt",options?:(Bacon|Condiments)[]};
type Bacon={amount:Optional,name:"Bacon"};
type Optional="Regular";
type Condiments={amount:Amount,name:"Mayo"};
type Amount="Regular"|"Extra";
type Burger=GenericBurger<"Bacon Double Cheeseburger"|"Bacon Cheeseburger"|"Double Cheeseburger"|"Cheeseburger">;
type GenericBurger<NAME>={name:NAME,options?:(Bacon|Condiments)[]};
type Chicken=GenericChicken<"Grilled Chicken Sandwich"|"Cordon Bleu">;
type GenericChicken<NAME>={name:NAME,options?:(Bacon|Condiments)[]};
type KoreanChicken={name:"Sweet and Spicy Chicken"|"Seasame Soy Chicken"|"Spicy Garlic 
Chicken",options?:(Bacon|Condiments)[]};
type Pitas={name:"Lemon Chicken Pita"|"Smokey Chicken Pita"|"Tangy Chicken Pita",options?:(Bacon|Condiments)[]};
type Fish={name:"Captain Nemo Burger",options?:(Bacon|Condiments)[]};
type ComboTwo={name:"Twofer Combo",one:TwoThreeChoices,two:TwoThreeChoices};
type TwoThreeChoices=GenericChicken<"Grilled Chicken Sandwich">|GenericBurger<"Bacon 
Cheeseburger">|FountainDrink<"Coca-Cola"|"Diet Coke"|"Dr. Pepper"|"Sprite","Medium">|CHOOSE;
type FountainDrink<NAME extends DrinkNames,SIZE extends DrinkSizes>={name:NAME,size:SIZE};
type DrinkNames="Coca-Cola"|"Coca-Cola Zero Sugar";
type DrinkSizes="CHOOSE";
type CHOOSE="CHOOSE";
type ComboThree={name:"Threefer Combo",one:TwoThreeChoices,two:TwoThreeChoices,three:TwoThreeChoices};
~~~

* Only use CHOOSE when the user didn't specify a choice and there is no template literal contraining the choice.
* If request doesn't match the schema exactly, choose the closest matching item that is strictly legal for the 
schema.
* Combos that don't specify enough items should use CHOOSE instead of duplicating an item.
* Return the JSON object of type `Cart`.
```

**assistant:**
```json
{
  "items": [
    {
      "name": "Grilled Chicken Sandwich"
    },
    {
      "name": "Captain Nemo Burger"
    },
    {
      "name": "Coca-Cola",
      "size": "CHOOSE"
    },
    {
      "name": "Coca-Cola",
      "size": "CHOOSE"
    }
  ]
}
```

**user:** _add bacon and extra mayo the chicken and 86 the fish_

**assistant:**
```json
{
  "items": [
    {
      "name": "Grilled Chicken Sandwich",
      "options": [
        {
          "amount": "Regular",
          "name": "Bacon"
        },
        {
          "amount": "Extra",
          "name": "Mayo"
        }
      ]
    },
    {
      "name": "Coca-Cola",
      "size": "CHOOSE"
    },
    {
      "name": "Coca-Cola",
      "size": "CHOOSE"
    }
  ]
}
```

**No repairs**

**Full query**:
* add bacon and extra mayo the chicken and 86 the fish
* CHOOSE
* Grilled Chicken Sandwich
* Coca-Cola
* Captain Nemo Burger

## Case: 7ee.0
**PASSED**  

Keywords:   
Complete menu tokens: 1296  
Input tokens: 490, output tokens: 76

**system:**
```json
Your job is to convert text-based restaurant orders into JSON data structures.
You never return a text answer. You always generate a JSON data structure.
The JSON should conform to the following typescript type definition for type `Cart`:

~~~typescript
type Cart={items:Item[]};
type Item=WiseguyMeal<ComboSizes>|Wiseguy|PattyMelt|Burger|KoreanChicken;
type WiseguyMeal<SIZE extends ComboSizes>={name:"Wiseguy 
Meal",size:SIZE,sandwich:Wiseguy|CHOOSE,fries:CHOOSE,drink:ChooseDrink<SIZE>};
type ComboSizes="CHOOSE";
type Wiseguy=GenericWiseguy<"Wiseguy"|"Vegan Wiseguy"|"Double Wiseguy"|"Triple Wiseguy"|"Down East Wiseguy">;
type GenericWiseguy<NAME>={name:NAME,type:"Regular"|"With Bacon"|"With Cheese"|"With Bacon and 
Cheese"|CHOOSE,options?:Veggies};
type CHOOSE="CHOOSE";
type Veggies={amount:ExtraAmount,name:"Tomato"};
type ExtraAmount="No"|"Regular"|"extra";
type ChooseDrink<SIZE extends DrinkSizes>=CHOOSE;
type DrinkSizes="CHOOSE";
type PattyMelt={name:"Mushroom and Swiss Melt",options?:(Veggies|Condiments)[]};
type Condiments={amount:Amount,name:"Mayo"};
type Amount="No"|"Regular"|"Extra";
type Burger=GenericBurger<"Bacon Double Cheeseburger"|"Bacon Cheeseburger"|"Double Cheeseburger"|"Cheeseburger">;
type GenericBurger<NAME>={name:NAME,options?:(Veggies|Condiments)[]};
type KoreanChicken={name:"Sweet and Spicy Chicken",options?:(Veggies|Condiments)[]};
~~~

* Only use CHOOSE when the user didn't specify a choice and there is no template literal contraining the choice.
* If request doesn't match the schema exactly, choose the closest matching item that is strictly legal for the 
schema.
* Combos that don't specify enough items should use CHOOSE instead of duplicating an item.
* Return the JSON object of type `Cart`.
```

**assistant:**
```json
{
  "items": []
}
```

**user:** _can I a double wiseguy with no tomatoes and extra mayo_

**assistant:**
```json
{
  "items": [
    {
      "name": "Double Wiseguy",
      "type": "CHOOSE",
      "options": [
        {
          "amount": "No",
          "name": "Tomato"
        },
        {
          "amount": "Extra",
          "name": "Mayo"
        }
      ]
    }
  ]
}
```

**No repairs**

**Full query**:
* can I a double wiseguy with no tomatoes and extra mayo
